In [1]:
import os
from typing import List
from pydantic import BaseModel
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.youtube import YoutubeLoader
from langchain_community.document_loaders import ArxivLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langgraph.graph import StateGraph, END

In [2]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
llm=init_chat_model("openai:gpt-4o-mini")

In [3]:
def load_text_retriever(file_path):
        docs = TextLoader(file_path, encoding="utf-8").load()
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = splitter.split_documents(docs)
        vs = FAISS.from_documents(chunks, OpenAIEmbeddings())
        return vs.as_retriever()
def load_youtube_retriever():
        # Mocked YouTube transcript text
        content = """
        This video explains how agentic AI systems rely on feedback loops, memory, and tool use.\n",
        It compares them to traditional pipeline-based LLMs. Temporal reasoning and autonomous tasking are emphasized.\n",
        """
        doc = Document(page_content=content, metadata={"source": "youtube"})
        vectorstore = FAISS.from_documents([doc], OpenAIEmbeddings())
        return vectorstore.as_retriever()
def wikipedia_search(query: str) -> str:
    print("🌐 Searching Wikipedia...")
    return WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())(query)
def arxiv_search(query: str) -> str:
    print("📄 Searching ArXiv...")
    results = ArxivLoader(query).load()
    return "".join(doc.page_content for doc in results[:2]) or "No relevant papers found."

In [4]:
text_retriever = load_text_retriever("sample_docs.txt")
youtube_retriever = load_youtube_retriever()

In [5]:
### state\n",
class MultiSourceRAGState(BaseModel):
    question: str
    text_docs: List[Document] = []
    yt_docs: List[Document] = []
    wiki_context: str = ""
    arxiv_context: str = ""
    final_answer: str = ""

In [6]:
### Retrieval Nodes\n",
def retrieve_text(state: MultiSourceRAGState) -> MultiSourceRAGState:
    docs = text_retriever.invoke(state.question)
    return state.model_copy(update={"text_docs": docs})
def retrieve_yt(state: MultiSourceRAGState) -> MultiSourceRAGState:
    docs = youtube_retriever.invoke(state.question)
    return state.model_copy(update={"yt_docs": docs})
def retrieve_wikipedia(state: MultiSourceRAGState) -> MultiSourceRAGState:
    result = wikipedia_search(state.question)
    return state.model_copy(update={"wiki_context": result})
def retrieve_arxiv(state: MultiSourceRAGState) -> MultiSourceRAGState:
    result = arxiv_search(state.question)
    return state.model_copy(update={"arxiv_context": result})

In [7]:
## synthesize\n",
def synthesize_answer(state: MultiSourceRAGState) -> MultiSourceRAGState:
    context = ""
    context += "[Internal Docs]" + "".join([doc.page_content for doc in state.text_docs])
    context += "[YouTube Transcript]" + "".join([doc.page_content for doc in state.yt_docs])
    context += "[Wikipedia]" + state.wiki_context
    context += "[ArXiv]" + state.arxiv_context
    prompt = f"""You have retrieved relevant context from multiple sources. Now synthesize a complete and coherent answer.\n",
    Question: {state.question}
    Context:
    {context}
    Final Answer:"""
    answer = llm.invoke(prompt).content.strip()
    return state.model_copy(update={"final_answer": answer})

In [9]:
builder = StateGraph(MultiSourceRAGState)
builder.add_node("retrieve_text", retrieve_text)
builder.add_node("retrieve_yt", retrieve_yt)
builder.add_node("retrieve_wiki", retrieve_wikipedia)
builder.add_node("retrieve_arxiv", retrieve_arxiv)
builder.add_node("synthesize", synthesize_answer)
builder.set_entry_point("retrieve_text")
builder.add_edge("retrieve_text", "retrieve_yt")
builder.add_edge("retrieve_yt", "retrieve_wiki")
builder.add_edge("retrieve_wiki", "retrieve_arxiv")
builder.add_edge("retrieve_arxiv", "synthesize")
builder.add_edge("synthesize", END)
graph = builder.compile()

In [ ]:
question = "What are transformer agents and how are they evolving in recent research?"
state = MultiSourceRAGState(question=question)
result = graph.invoke(state)
print("✅ Final Answer:")
print(result["final_answer"])

ValidationError: 1 validation error for MultiSourceRAGState
question
  Input should be a valid string [type=string_type, input_value=('What are transformer ag...n recent research?"\n',), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type